In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
if 'google.colab' in str(get_ipython()): # running in colab
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = '/content/drive/MyDrive/revisiting_crossval_RNAfolding/'
else:
    DATA_PATH = '../shared_insync/revisiting_crossval_RNAfolding/'

## Computing performance scores

Scoring functions

In [3]:
def get_tp(bp_x, bp_ref, strict):
    tp = 0
    for rbp in bp_x:
        cond = rbp in bp_ref
        if not strict:
            cond = cond or [rbp[0], rbp[1] - 1] in bp_ref or [rbp[0], rbp[1] + 1] in bp_ref or [rbp[0] + 1, rbp[1]] in bp_ref or [rbp[0] - 1, rbp[1]] in bp_ref
        if cond:
            tp += 1
    return tp   

def bp_metrics(ref_bp, pre_bp, strict=True):
    """F1, recall and precision score from base pairs. Is the same as triangular but less efficient. strict=False takes into account the  neighbors for each base pair as correct"""
    assert all([type(bp) == list for bp in ref_bp]), "ref_bp must be a list of lists"
    assert all([type(bp) == list for bp in pre_bp]), "pre_bp must be a list of lists"

    # corner case when there are no positives
    if len(ref_bp) == 0 and len(pre_bp) == 0:
        return 1.0, 1.0, 1.0

    tp1 = get_tp(pre_bp, ref_bp, strict)
    tp2 = get_tp(ref_bp, pre_bp, strict)

    fn = len(ref_bp) - tp1
    fp = len(pre_bp) - tp1

    tpr = pre = f1 = 0.0
    if tp1 + fn > 0:
        tpr = tp1 / float(tp1 + fn)  # sensitivity (=recall =power)
    if tp1 + fp > 0:
        pre = tp2 / float(tp1 + fp)  # precision (=ppv)
    if tpr + pre > 0:
        f1 = 2 * pre * tpr / (pre + tpr)  # F1 score

    return f1, tpr, pre


Reference dataset

In [4]:
dataset = pd.read_csv(f"{DATA_PATH}data/archiveII_250808.csv", index_col="id")
dataset.head()

,sequence,len,structure,base_pairs
id,,,,
5s_Acholeplasma-laidlawii-1,UCUGGUGACGAUAGGUAAGAUGGUUCACCUGUUCCCAUCCCGAACA...,112,((((((((......((((((((....((((((.............)...,"[[1, 111], [2, 110], [3, 109], [4, 108], [5, 1..."
5s_Acidovorax-temperans-1,UGCCUGAUGACCAUAGCAAGUUGGUACCACUCCUUCCCAUCCCGAA...,115,.(((((((((.....((((((((.....((((((...............,"[[2, 115], [3, 114], [4, 113], [5, 112], [6, 1..."
tmRNA_Stre.gord._TRW-29390_1-349,GGGGUCGUUACGGAUUCGACAGGCAUUAUGAGGCAUAUUUUGCGAC...,349,(((((((............((((((((....(((((((((..((((...,"[[1, 345], [2, 344], [3, 343], [4, 342], [5, 3..."
tRNA_tdbR00000055-Schizosaccharomyces_pombe-4896-Glu-3UC,UCCGUUGUGGUCCAACGGCUAGGAUUCGUCGCUUUCACCGACGGGA...,75,(((((((..((((........))))((((((.......)))))).....,"[[1, 71], [2, 70], [3, 69], [4, 68], [5, 67], ..."
srp_List.mono._U15684,UGGGUUGAUGAGCGUGAAGCCUUCGCUCGGUUGGAUUUUUCUUCAU...,279,.(.((((...(.(.((.(.((..(.....)..)).)...(...(.....,"[[2, 276], [4, 274], [5, 273], [6, 272], [7, 2..."


Classical methods are not trained, therefore the score per sequence is the same in all cases

In [5]:
classical_methods = ["IPKnot", "LinearFoldV", "LinearPartitionC", "LinearPartitionV", "ProbKnot", "RNAfold", "RNAstructure"] 
trained_methods = ["MxFold2", "REDfold", "UFold", "sincFold"]

# load predictions and compute f1 scores
classical_summary = []
for method in classical_methods:
    print(method, end=" ")
    pred = pd.read_csv(DATA_PATH+f"results/{method}.csv", index_col="id")
    pred["ref"] = dataset.loc[pred.index, "base_pairs"]
    pred[["f1", "rec", "pre"]] = pred.apply(lambda x: bp_metrics(json.loads(x["ref"]), json.loads(x["base_pairs"])), axis=1, result_type="expand")
    pred["method"] = method
    pred = pred[["method", "f1", "rec", "pre"]]
    classical_summary.append(pred)
classical_summary = pd.concat(classical_summary)    

IPKnot LinearFoldV LinearPartitionC LinearPartitionV ProbKnot RNAfold RNAstructure 

### Random k-fold

In [ ]:
partition_name = "1_random_kfolds"
splits = pd.read_csv(f"{DATA_PATH}data/{partition_name}/split.csv", index_col="id")
splits.groupby("fold_number").count()

,partition
fold_number,
0,3864
1,3864
2,3864
3,3864
4,3864


In [ ]:
import os

folds = sorted(splits["fold_number"].unique())
summary = []
for fold in folds:
    partition_set = splits[(splits["fold_number"] == fold) & (splits["partition"] == "test")]
    print(fold, end=" ")
    for method in trained_methods:
        f = DATA_PATH+f"results/{partition_name}/{method}/{fold}/pred.csv"
        if not os.path.exists(f):
            print(f"\n{method, fold} not found, skipping\n")
            continue
        print(method, end=" ")
        
        pred = pd.read_csv(f, index_col="id")
        pred["ref"] = dataset.loc[partition_set.index, "base_pairs"]
        pred[["f1", "rec", "pre"]] = pred.apply(lambda x: bp_metrics(json.loads(x["ref"]), json.loads(x["base_pairs"])), axis=1, result_type="expand")
        pred["method"] = method
        pred["fold"] = fold
        pred = pred[["method", "fold", "f1", "rec", "pre"]]
        summary.append(pred)
    print()
summary = pd.concat(summary)   

0 MxFold2 REDfold UFold sincFold 
1 MxFold2 REDfold UFold sincFold 
2 MxFold2 REDfold UFold sincFold 
3 MxFold2 REDfold UFold sincFold 
4 MxFold2 REDfold UFold sincFold 


In [ ]:
# add classical methods to kfold summary
for method in classical_methods:
    for fold in folds:
        partition_set = splits[(splits["fold_number"] == fold) & (splits["partition"] == "test")]
        res = classical_summary.loc[partition_set.index]
        res = res[res["method"] == method]
        res["fold"] = fold
        summary = pd.concat([summary, res])

os.makedirs("results", exist_ok=True) 
summary.to_csv(f"results/{partition_name}.csv")

array(['MxFold2', 'REDfold', 'UFold', 'sincFold', 'IPKnot', 'LinearFoldV',
       'LinearPartitionC', 'LinearPartitionV', 'ProbKnot', 'RNAfold',
       'RNAstructure'], dtype=object)

### Fam-fold

In [ ]:
partition_name = "2_fam_folds"
splits = pd.read_csv(f"{DATA_PATH}data/{partition_name}/split.csv", index_col="id")

In [ ]:
folds = sorted(splits["fold_number"].unique())
summary = []
for fold in folds:
    partition_set = splits[(splits["fold_number"] == fold) & (splits["partition"] == "test")]
    print(fold, end=" ")
    for method in trained_methods:
        f = DATA_PATH+f"results/{partition_name}/{method}/{fold}/pred.csv"
        if not os.path.exists(f):
            print(f"\n{method, fold} not found, skipping\n")
            continue
        print(method, end=" ")
        
        pred = pd.read_csv(f, index_col="id")
        pred["ref"] = dataset.loc[partition_set.index, "base_pairs"]
        pred[["f1", "rec", "pre"]] = pred.apply(lambda x: bp_metrics(json.loads(x["ref"]), json.loads(x["base_pairs"])), axis=1, result_type="expand")
        pred["method"] = method
        pred["fold"] = fold
        pred = pred[["method", "fold", "f1", "rec", "pre"]]
        summary.append(pred)
    print()
summary = pd.concat(summary)    

0 MxFold2 REDfold UFold sincFold 
1 MxFold2 REDfold UFold sincFold 
2 MxFold2 REDfold UFold sincFold 
3 MxFold2 REDfold UFold sincFold 
4 MxFold2 REDfold UFold sincFold 
5 MxFold2 REDfold UFold sincFold 
6 MxFold2 REDfold UFold sincFold 
7 MxFold2 REDfold UFold sincFold 
8 MxFold2 REDfold UFold sincFold 


In [ ]:
# add classical methods to the summary
for method in classical_methods:
    for fold in folds:
        partition_set = splits[(splits["fold_number"] == fold) & (splits["partition"] == "test")]
        res = classical_summary.loc[partition_set.index]
        res = res[res["method"] == method]
        res["fold"] = fold
        summary = pd.concat([summary, res])
os.makedirs("results", exist_ok=True) 
summary.to_csv(f"results/{partition_name}.csv")   

### Clustering folds

In [17]:
partition_name = "3_clustering_folds"
splits = pd.read_csv(f"{DATA_PATH}data/{partition_name}/split.csv", index_col="id")

In [18]:
folds = sorted(splits["fold_number"].unique())
summary = []
for fold in folds:
    partition_set = splits[(splits["fold_number"] == fold) & (splits["partition"] == "test")]
    print(fold, end=" ")
    for method in trained_methods:
        f = DATA_PATH+f"results/{partition_name}/{method}/{fold}/pred.csv"
        if not os.path.exists(f):
            print(f"\n{method, fold} not found, skipping\n")
            continue
        print(method, end=" ")
        
        pred = pd.read_csv(f, index_col="id")
        pred["ref"] = dataset.loc[partition_set.index, "base_pairs"]
        pred[["f1", "rec", "pre"]] = pred.apply(lambda x: bp_metrics(json.loads(x["ref"]), json.loads(x["base_pairs"])), axis=1, result_type="expand")
        pred["method"] = method
        pred["fold"] = fold
        pred = pred[["method", "fold", "f1", "rec", "pre"]]
        summary.append(pred)
    print()
summary = pd.concat(summary)    

0 MxFold2 REDfold UFold sincFold 
1 MxFold2 REDfold UFold sincFold 
2 MxFold2 REDfold UFold sincFold 
3 MxFold2 REDfold UFold sincFold 
4 MxFold2 REDfold UFold sincFold 


In [19]:
# add classical methods to the summary
for method in classical_methods:
    for fold in folds:
        partition_set = splits[(splits["fold_number"] == fold) & (splits["partition"] == "test")]
        res = classical_summary.loc[partition_set.index]
        res = res[res["method"] == method]
        res["fold"] = fold
        summary = pd.concat([summary, res])
os.makedirs("results", exist_ok=True) 
summary.to_csv(f"results/{partition_name}.csv")   

### HL folds
- sincFold TODO
- mxfold2 h5 missing?

In [8]:
partition_name = "5_hl_folds"
splits = pd.read_csv(f"{DATA_PATH}data/{partition_name}/split.csv", index_col="id")

In [9]:
folds = sorted(splits["fold_name"].unique())
summary = []
for fold in folds:
    partition_set = splits[(splits["fold_name"] == fold) & (splits["partition"] == "test")]
    print(fold, end=" ")
    for method in trained_methods:
        f = DATA_PATH+f"results/{partition_name}/{method}/{fold}/pred.csv"
        if not os.path.exists(f):
            print(f"\n{method, fold} not found, skipping\n")
            continue
        print(method, end=" ")
        
        pred = pd.read_csv(f, index_col="id")
        pred["ref"] = dataset.loc[partition_set.index, "base_pairs"]
        pred[["f1", "rec", "pre"]] = pred.apply(lambda x: bp_metrics(json.loads(x["ref"]), json.loads(x["base_pairs"])), axis=1, result_type="expand")
        pred["method"] = method
        pred["fold"] = fold
        pred = pred[["method", "fold", "f1", "rec", "pre"]]
        summary.append(pred)
    print()
summary = pd.concat(summary)    

hl10 MxFold2 REDfold UFold sincFold 
hl15 MxFold2 REDfold UFold sincFold 
hl20 MxFold2 REDfold UFold sincFold 
hl25 MxFold2 REDfold UFold sincFold 
hl30 MxFold2 REDfold UFold sincFold 
hl35 MxFold2 REDfold UFold sincFold 
hl40 MxFold2 REDfold UFold sincFold 
hl45 MxFold2 REDfold UFold sincFold 
hl50 MxFold2 REDfold UFold sincFold 
hl55 MxFold2 REDfold UFold sincFold 
hl60 MxFold2 REDfold UFold sincFold 
hl65 MxFold2 REDfold UFold sincFold 
hl70 MxFold2 REDfold UFold sincFold 
hl75 MxFold2 REDfold UFold sincFold 
hl80 MxFold2 REDfold UFold sincFold 
hl85 MxFold2 REDfold UFold sincFold 
hl90 MxFold2 REDfold UFold sincFold 
hl95 MxFold2 REDfold UFold sincFold 


In [10]:
# add classical methods to the summary
for method in classical_methods:
    for fold in folds:
        partition_set = splits[(splits["fold_name"] == fold) & (splits["partition"] == "test")]
        res = classical_summary.loc[partition_set.index]
        res = res[res["method"] == method]
        res["fold"] = fold
        summary = pd.concat([summary, res])
summary.method.unique()    
os.makedirs("results", exist_ok=True) 
summary.to_csv(f"results/{partition_name}.csv")       

### SIM folds


In [32]:
partition_name = "6_sim_folds"
splits = []
for f in os.listdir(f"{DATA_PATH}data/6_sim_folds/bootstrap/"):
    split = pd.read_csv(f"{DATA_PATH}data/6_sim_folds/bootstrap/{f}")
    split["similarity"] = f.split("_")[-1].split("-")[0]                    
    splits.append(split) 
splits = pd.concat(splits)
splits.head()

,fold_name,fold_number,partition,id,similarity
0,0,0,train,grp1_a.I1.m.M.grisea.B2.ND1,sim70
1,0,0,train,srp_Pyro.aero._AE009852,sim70
2,0,0,train,RNaseP_vHge3-5,sim70
3,0,0,train,telomerase_AF221924.105-538,sim70
4,0,0,train,srp_Mono.brev._GSP-81824,sim70


In [35]:
similarities = sorted(splits["similarity"].unique())
folds = sorted(splits["fold_number"].unique())
summary = []
for similarity in similarities:
    print(similarity, end=" ")
    for fold in folds:
        partition_set = splits[(splits["fold_number"] == fold) & (splits["similarity"] == similarity) & (splits["partition"] == "test")]
        print(fold, end=" ")
        for method in trained_methods:
            f = DATA_PATH+f"results/{partition_name}/{method}/{similarity}/{fold}/pred.csv"
            if not os.path.exists(f):
                print(f"\n{method, fold} not found, skipping\n")
                continue
            print(method, end=" ")
            
            pred = pd.read_csv(f, index_col="id")
            pred["ref"] = dataset.loc[partition_set.index, "base_pairs"]
            pred[["f1", "rec", "pre"]] = pred.apply(lambda x: bp_metrics(json.loads(x["ref"]), json.loads(x["base_pairs"])), axis=1, result_type="expand")
            pred["similarity"] = similarity
            pred["method"] = method
            pred["fold"] = fold
            pred = pred[["method", "similarity", "fold", "f1", "rec", "pre"]]
            summary.append(pred)
        print()
summary = pd.concat(summary)    
summary.head()

sim30 0 MxFold2 

KeyError: "None of [Index([4], dtype='int64')] are in the [index]"